Датасет о 1215 покемонах и их характеристиках. <br>
В датасете представлены следующие атрибуты:
- images: Путь к изображению покемона (строковой тип. В данной работе не используется)
- index: Номер покемона в Покедексе (числовой тип. Был удалён из датасета, так как используется стандартная индексация).
- Name: Название Покемона (строковой тип).
- Type 1: Первый из типов Покемона (строковой тип).
- Type 2: Второй из типов Покемона, если присутствует (строковой тип).
- Total: Сумма всех характеристик Покемона (числовой тип).
- HP: Очки здоровья Покемона (числовой тип).
- Attack: Показатель базовой атаки Покемона для обычных атак (числовой тип).
- Defense: Базовый показатель защиты Покемона от обычных атак (числовой тип).
- SP. Atk.: Показатель базовой атаки Покемона для особых атак (числовой тип).
- SP. Def.: Базовый показатель защиты Покемона от особых атак (числовой тип).
- Speed: Определяет, какой Покемон атакует первым в раунде (числовой тип).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from io import StringIO
import pydotplus
from ipywidgets import Image
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('pokedex.csv')
#Удаляем столбец image за ненадобностью
df.drop('Image', axis=1, inplace=True)
#Удаляем столбец index за ненадобностью
df.drop('Index', axis=1, inplace=True)
df

,Name,Type 1,Type 2,Total,HP,Attack,Defense,SP. Atk.,SP. Def,Speed
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,Venusaur Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,Charmander,Fire,NaN,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1210,Iron Crown,Steel,Psychic,590,90,72,100,122,108,98
1211,Terapagos Normal Form,Normal,NaN,450,90,65,85,65,85,60
1212,Terapagos Terastal Form,Normal,NaN,600,95,95,110,105,110,85
1213,Terapagos Stellar Form,Normal,NaN,700,160,105,110,130,110,85


In [2]:
df.isnull().sum()

Name          0
Type 1        0
Type 2      546
Total         0
HP            0
Attack        0
Defense       0
SP. Atk.      0
SP. Def       0
Speed         0
dtype: int64

In [3]:
df.fillna({'Type 2':'None'}, inplace=True)
df

,Name,Type 1,Type 2,Total,HP,Attack,Defense,SP. Atk.,SP. Def,Speed
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,Venusaur Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,Charmander,Fire,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1210,Iron Crown,Steel,Psychic,590,90,72,100,122,108,98
1211,Terapagos Normal Form,Normal,None,450,90,65,85,65,85,60
1212,Terapagos Terastal Form,Normal,None,600,95,95,110,105,110,85
1213,Terapagos Stellar Form,Normal,None,700,160,105,110,130,110,85


In [4]:
df.duplicated().sum()

0

In [5]:
numdf = df.select_dtypes(include=[np.number])

In [6]:
for i in list(numdf.columns):
    Q1=df[i].quantile(.25)
    Q3=df[i].quantile(.75)
    InterQ=Q3-Q1
    Q1-=InterQ*1.5
    Q3+=InterQ*1.5
    df = df[(df[i] >= Q1) & (df[i] <= Q3)]
df

,Name,Type 1,Type 2,Total,HP,Attack,Defense,SP. Atk.,SP. Def,Speed
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,Venusaur Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,Charmander,Fire,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1208,Raging Bolt,Electric,Dragon,590,125,73,91,137,89,75
1209,Iron Boulder,Rock,Psychic,590,90,120,80,68,108,124
1210,Iron Crown,Steel,Psychic,590,90,72,100,122,108,98
1211,Terapagos Normal Form,Normal,None,450,90,65,85,65,85,60


In [7]:
numdf = df.select_dtypes(include=[np.number])

Создаём map для столбца Type 1. Это необходимо для ROC оценки, так как в столбц больше 2 уникальных данных.

In [8]:
type_mapping = {'Grass': 0, 'Fire': 1, 'Water': 2, 'Bug': 3, 'Normal': 4, 'Dark': 5, 'Poison': 6, 'Electric': 7, 'Ground': 8, 'Ice': 9, 'Fairy': 10, 'Steel': 11, 'Fighting': 12, 'Psychic': 13, 'Rock': 14, 'Ghost': 15, 'Dragon': 16, 'Flying': 17}
df['Type 1'] = df['Type 1'].map(type_mapping)

Столбец Type 1 указываем, как целевой класс

In [9]:
df_y = df['Type 1']
df_x = numdf
df_x.shape, df_y.shape

((1136, 7), (1136,))

In [10]:
from sklearn.model_selection import cross_val_score, train_test_split

df_x_train, df_x_valid, df_y_train, df_y_valid = train_test_split(df_x, df_y, test_size=0.3, random_state=17)

In [11]:
df_x_train.shape, df_x_valid.shape

((795, 7), (341, 7))

In [12]:
df_y_train.shape, df_y_valid.shape

((795,), (341,))

Создаём дерево решений, основанное на энтропии

In [13]:
tree_entr = DecisionTreeClassifier(criterion="entropy")
tree_entr.fit(df_x_train, df_y_train)

DecisionTreeClassifier(criterion='entropy')

Точность предсказаний:

In [14]:
tree_entr.score(df_x_valid,df_y_valid)

0.187683284457478

Графический вывод дерева

In [15]:
dot_data=StringIO()
export_graphviz(tree_entr, feature_names=df_x.columns, out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.31873 to fit



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x7f\xff\x00\x00\x03\x16\x08\x02\x00\x00\x00\xdc\x13a…

Дерево вышло настроенным на тестовую выборку, поэтому перебалансируем дерево

In [36]:
tree_params = {"max_depth": np.arange(1, 8), "max_features": [0.5, 0.7, 1]}
tree_grid = GridSearchCV(tree_entr, tree_params, cv=5, n_jobs=-1)
tree_grid.fit(df_x_train,df_y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(criterion='entropy'),
             n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7]),
                         'max_features': [0.5, 0.7, 1]})

In [37]:
dot_data=StringIO()
export_graphviz(tree_grid.best_estimator_, feature_names=df_x.columns, out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())

Image(value=b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x18\xcf\x00\x00\x02\xe0\x08\x06\x00\x00\x00'\xd1\x06…

In [38]:
tree_grid.best_score_, tree_grid.best_params_

(0.16855345911949687, {'max_depth': 4, 'max_features': 0.5})

Реализуем алгоритм knn:

In [19]:
knn = KNeighborsClassifier()
knn.fit(df_x_train, df_y_train)

KNeighborsClassifier()

In [20]:
knn.score(df_x_valid,df_y_valid)

0.17302052785923755

Аналогично перебалансируем knn

In [21]:
knn_params = {"n_neighbors": [1, 2, 3, 4] + list(range(5, 30, 5))}
knn_grid = GridSearchCV(knn, knn_params, cv=5)
knn_grid.fit(df_x_train, df_y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 10, 15, 20, 25]})

In [22]:
knn_grid.best_score_, knn_grid.best_params_

(0.18364779874213835, {'n_neighbors': 20})

In [23]:
tree_valid_pred = tree_grid.predict(df_x_valid)
knn_valid_pred = knn_grid.predict(df_x_valid)

Оценка по accuracy

In [24]:
accuracy_score(df_y_valid, tree_valid_pred)

0.18181818181818182

In [25]:
accuracy_score(df_y_valid, knn_valid_pred)

0.20234604105571846

Оценка по precision

In [26]:
precision_score(df_y_valid, tree_valid_pred, average='micro')

0.18181818181818182

In [27]:
precision_score(df_y_valid, knn_valid_pred, average='micro')

0.20234604105571846

Оценка по recall

In [28]:
recall_score(df_y_valid, tree_valid_pred, average='micro')

0.18181818181818182

In [29]:
recall_score(df_y_valid, knn_valid_pred, average='micro')

0.20234604105571846

Оценка по f-measure

In [30]:
f1_score(df_y_valid, tree_valid_pred, average='micro')

0.18181818181818182

In [31]:
f1_score(df_y_valid, knn_valid_pred, average='micro')

0.20234604105571846

Оценка по площади под ROC кривой

In [32]:
tree_pred_proba = tree_grid.predict_proba(df_x_valid)
roc_auc_score(df_y_valid, tree_pred_proba, multi_class='ovr')

0.5812750157265034

In [33]:
knn_pred_proba = knn_grid.predict_proba(df_x_valid)
roc_auc_score(df_y_valid, knn_pred_proba, multi_class='ovr')

0.6388528395091086